### Import des bibliotheques utiles et des classes

In [22]:
import pandas as pd
import re 
from Corpus import Corpus
from Authors import Author
from SearchEngine import SearchEngine
%matplotlib inline
# Pour les graphiques interactifs dans Jupyter Notebook


# TD 8 :

Cette partie v3 ne contient que le notebook, les objets créés au fil des TDs et les fichiers de test. Les fonctions des TDs précédents sont présentes dans les anciennes versions (V1 et 2).

# Partie 1

In [23]:
discours = pd.read_csv('discours_US.csv', sep='\t')

## 1.2 Répartition des auteurs dans le fichier :

In [24]:
discours["speaker"].value_counts()

speaker
CLINTON    93
TRUMP      71
Name: count, dtype: int64

## 1.3 Création d'un corpus à partir du csv Discours_US :

modification réalisé du constructeur corpus pour qu'il prenne directement les dictionnaire à l'instanciation (par rapport à la v2)

Le corpus est conçu pour fonctionner avec des objets documents, nous allons donc recréé un objet hérité de Document pour ce fichier

In [25]:
from Document import Document
class Discours(Document):
    def __init__(self, authors, text, published, link, source, description):
        super().__init__("", authors, text, published, link, source)
        self._description = description
#getter description
    @property
    def description(self):
        return self._description
#setter description
    @description.setter
    def description(self, description):
        self._description = description
    @property
    def authors(self):
        return str(self._authors)
    @authors.setter
    def authors(self, value):
        if isinstance(value, str):
            self._authors = value


Nous allons créé les dictionnaires de document et d'auteurs pour alimenter le corpus (en découpant les textes en phrases pour multiplier les documents)

In [26]:
id2doc = {}
id2author = {}
ligne_id = 0
# doc : title, author, text, published 
# authors : doc['author'] = objet Author(name=author_name) --> aut puis aut.addpublication(id_doc) 
# On découpe le texte en phrase qui représentera 1 document
source = "discours_US"
for index, row in discours.iterrows():
    published = pd.to_datetime(row['date'], errors='coerce')
    author_name = row['speaker']
    link = row['link']
    description = row['descr']
    textes = row['text']
    textes = re.split(r'[.!?]+', textes) # découpage en phrases avec regex
    textes = [text.strip() for text in textes if text.strip()] # nettoyage des espaces vides
    for phrase in textes:
    # conversion de published en datetime
        id2doc[ligne_id] = Discours(
            authors=author_name,
            text=phrase, 
            published=published, 
            link=link, 
            source=source, 
            description=description)
        if author_name not in id2author:
            id2author[author_name] = Author(name=author_name)
        id2author[author_name].add_publication(ligne_id)
        ligne_id += 1


sujet = "discours_US"
corpus = Corpus(sujet, id2doc, id2author)
print(f"Corpus '{sujet}' créé avec {len(id2doc)} documents et {len(id2author)} auteurs.")

Corpus 'discours_US' créé avec 33836 documents et 2 auteurs.


## 1.4 Test du corpus avec les fonctions search et concorde :

In [27]:
corpus.search("america")

['America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'America',
 'Am

In [28]:
corpus.concorde("america")

,contexteGauche,keyword,contexteDroite
0,"use when families are strong,",america,is strong So I'm hitting the
1,esses starting and growing in,america,again We have stalled out I w
2,ths about race and justice in,america,There is something profoundly
3,ne this shared vision of what,america,can be and should be I learne
4,sweeping small-town and rural,america,as well We have to do more an
...,...,...,...
1240,ign You represent the best of,america,and being your candidate has
1241,we love and about building an,america,"that's hopeful, inclusive and"
1242,hought But I still believe in,america,and I always will And if you
1243,"that better, stronger, fairer",america,we seek And I know you will I


# Partie 2 : Utilisation du moteur de recherche

## 2.1 Initialisation de search_engine

In [29]:
search_engine = SearchEngine(corpus)

## 2.2 Test de la fonction de recherche avec plusieurs exemples :

In [30]:
test_mots_cles = ["america", "democracy", "economy", "war", "peace"]
for mot_cle in test_mots_cles:
    print(f"\nRecherche pour le mot-clé : '{mot_cle}'")
    print(search_engine.search(mot_cle, nb_doc_retour=3))


Recherche pour le mot-clé : 'america'
       doc_id              similarity
13390   13390                 [[1.0]]
13389   13389  [[0.7954958640429435]]
10964   10964  [[0.7954958640429435]]

Recherche pour le mot-clé : 'democracy'
       doc_id               similarity
33408   33408   [[0.4979750672935294]]
28659   28659   [[0.3727918029240991]]
21228   21228  [[0.36703384269282147]]

Recherche pour le mot-clé : 'economy'
       doc_id              similarity
19957   19957  [[0.6010001440254502]]
3482     3482  [[0.5826537973337758]]
8747     8747  [[0.5389856755677461]]

Recherche pour le mot-clé : 'war'
       doc_id               similarity
9727     9727                  [[1.0]]
9791     9791   [[0.5194151669306858]]
20626   20626  [[0.45610133620740945]]

Recherche pour le mot-clé : 'peace'
       doc_id              similarity
10925   10925  [[0.5931564558491165]]
24772   24772  [[0.4472135954999579]]
9373     9373  [[0.4039664512995502]]


## 2.3 Ajout d'un compteur à la fonction de recherche (search2 créé pour garder les réponses au dessus)

In [31]:
from SearchEngine import SearchEngine
search_engine = SearchEngine(corpus)

In [32]:
for mot_cle in test_mots_cles:
    print(f"\nRecherche pour le mot-clé : '{mot_cle}'")
    search_engine.search2(mot_cle, nb_doc_retour=500)


Recherche pour le mot-clé : 'america'


Calcul des similarités pour le mot 'america': 100%|██████████| 33836/33836 [00:03<00:00, 10354.65doc/s]



Recherche pour le mot-clé : 'democracy'


Calcul des similarités pour le mot 'democracy': 100%|██████████| 33836/33836 [00:02<00:00, 13504.87doc/s]



Recherche pour le mot-clé : 'economy'


Calcul des similarités pour le mot 'economy': 100%|██████████| 33836/33836 [00:02<00:00, 13438.81doc/s]



Recherche pour le mot-clé : 'war'


Calcul des similarités pour le mot 'war': 100%|██████████| 33836/33836 [00:02<00:00, 14519.05doc/s]



Recherche pour le mot-clé : 'peace'


Calcul des similarités pour le mot 'peace': 100%|██████████| 33836/33836 [00:02<00:00, 14069.61doc/s]


In [33]:
search_engine.search2("america freedom", nb_doc_retour=5)

Calcul des similarités pour le mot 'freedom': 100%|██████████| 33836/33836 [00:02<00:00, 13987.73doc/s]


,doc_id,similarity
6081,6081,[[1.0]]
13390,13390,[[1.0]]
13389,13389,[[0.7954958640429435]]
13392,13392,[[0.7954958640429435]]
10964,10964,[[0.7954958640429435]]


# TD 9_10 Interface graphique

## Configuration :

Bibliotheques ipywidget pour un affichage interactif :

In [34]:
from ipywidgets import Text, Button, Output
from IPython.display import display

Création des différents widgets :

In [35]:
# Champ de saisie de texte pour le mot à rechercher
texte = Text(description='Entrez un mot à rechercher:', placeholder='Cliquez ici...', layout={'width': '400px'}, style={'description_width': 'initial'})
# Champ de saisie pour le nombre de documents à retourner
text_nb_doc = Text(description='Nombre de documents à retourner:', value='5', layout={'width': '400px'}, style={'description_width': 'initial'})
# Bouton de recherche
bouton_recherche = Button(description='Rechercher')

# Bouton pour afficher le graphique de l'évolution temporelle des occurences des mots recherchés
bouton_graph = Button(description="Afficher l'évolution temporelle des mots recherchés", button_style='success', layout={'width': '400px'})

# bouton de retour (qui va relancer la recherche pour le moment)
bouton_retour = Button(description="retour à la recherche")


# Zone de sortie
output = Output()


Fonction de recherche :

In [36]:
# Fonction appelée quand on clique le bouton de recherche
def recherche(b, search_engine=search_engine, nb_doc=text_nb_doc):
    with output:
        # Effacer la sortie précédente
        output.clear_output()
        # Effectuer la recherche avec search2 (pour la barre de progression)
        recherche = search_engine.search2(texte.value, nb_doc_retour=int(nb_doc.value))

        # Pour chaque document trouvé, afficher les détails
        for doc_id in recherche.index:
            print(f"\n Document correspondant :  \n" , f"Auteur(s) : {search_engine.corpus.id2document[doc_id].authors}")
            print(f"Texte: {search_engine.corpus.id2document[doc_id].text}")


            # Créer un bouton pour afficher les statistiques détaillées du document
            bouton_stats = Button(description=f"Détails du document : {doc_id}", 
                                  button_style='info', 
                                  layout={'width': '200px'},
                                  value = doc_id)
            bouton_stats.doc_id = doc_id # on attache l'id du doc au bouton
            bouton_stats.on_click(stats_docs)
            display(bouton_stats)
    

Fonction d'affichage des détails d'un document :

In [37]:
# Fonction appelée quand on clique le bouton statistiques
def stats_docs(b):
    with output:
        output.clear_output()
        doc_id = int(b.doc_id)
        doc = search_engine.corpus.id2document[doc_id]
        print(f"Détails pour le Document ID: {doc_id}")
        print(f"Auteur : {doc.authors}")
        print(f"Publié le: {doc.published}")
        print(f"Lien: {doc.link}")
        print(f"Source: {doc.source}")
        print(f"Texte: {doc.text}")
        # Attribut créé pour Discours
        if hasattr(doc, 'description'):
            print(f"Description: {doc.description}")
            
        display(bouton_retour)


Fonction d'affichage du graphique de l'évolution temporelle des mots recherchés : 

In [38]:
# Fonction pour le bouton afficher évolution
def afficher_evolution(b):
    with output:
        output.clear_output()
        search_engine.evolution_presence_mot(texte.value.split())
        display(bouton_retour)

Association des fonctions aux boutons :

In [39]:
bouton_recherche.on_click(recherche)
bouton_graph.on_click(afficher_evolution)
bouton_retour.on_click(recherche)

## Affichage :

Les dates ne sont pas encore gérés sur les documents Arxvis et Reddit, le graph ne peut donc pas se faire :

## 

In [40]:

if type(corpus.id2document[1]) == Discours:
    display(texte, text_nb_doc, bouton_recherche, bouton_graph, output)
else:
    display(texte, text_nb_doc, bouton_recherche, output)




Text(value='', description='Entrez un mot à rechercher:', layout=Layout(width='400px'), placeholder='Cliquez i…

Text(value='5', description='Nombre de documents à retourner:', layout=Layout(width='400px'), style=TextStyle(…

Button(description='Rechercher', style=ButtonStyle())

Output()